<a href="https://colab.research.google.com/github/nekroz-coder/MachineLearning/blob/main/week13/CNN_Modern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://poloclub.github.io/cnn-explainer/


https://convnetplayground.fastforwardlabs.com/#/



LeNet-5, AlexNet and VGGNet.




https://landing.ai/
https://www.ultralytics.com/

# Multi Layer Perceptron

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 1: Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 2: Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 300)
        self.fc2 = nn.Linear(300, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MLP()

# Step 3: Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Step 5: Evaluate the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training and Evaluation
for epoch in range(1, 11): # Train for 10 epochs
    train(model, device, train_loader, optimizer, epoch)
    test_loss, correct = test(model, device, test_loader)
    print(f"Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


100%|██████████| 9912422/9912422 [00:00<00:00, 162382921.26it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 116701053.78it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 55327573.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20223491.26it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1: Test Loss: 0.0026, Accuracy: 9495/10000
Epoch 2: Test Loss: 0.0019, Accuracy: 9626/10000
Epoch 3: Test Loss: 0.0019, Accuracy: 9616/10000
Epoch 4: Test Loss: 0.0017, Accuracy: 9649/10000
Epoch 5: Test Loss: 0.0015, Accuracy: 9694/10000
Epoch 6: Test Loss: 0.0014, Accuracy: 9750/10000
Epoch 7: Test Loss: 0.0012, Accuracy: 9790/10000
Epoch 8: Test Loss: 0.0013, Accuracy: 9760/10000
Epoch 9: Test Loss: 0.0014, Accuracy: 9736/10000
Epoch 10: Test Loss: 0.0015, Accuracy: 9703/10000


#LeNet5

In [ ]:

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.Resize((32, 32)), # For LeNet
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct


In [ ]:
# Example for LeNet-5
model = LeNet5().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, correct = test(model, device, test_loader, criterion)
    print(f"LeNet-5 - Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


LeNet-5 - Epoch 1: Test Loss: 0.0011, Accuracy: 9791/10000
LeNet-5 - Epoch 2: Test Loss: 0.0006, Accuracy: 9868/10000
LeNet-5 - Epoch 3: Test Loss: 0.0007, Accuracy: 9864/10000
LeNet-5 - Epoch 4: Test Loss: 0.0008, Accuracy: 9837/10000
LeNet-5 - Epoch 5: Test Loss: 0.0005, Accuracy: 9887/10000
LeNet-5 - Epoch 6: Test Loss: 0.0005, Accuracy: 9907/10000
LeNet-5 - Epoch 7: Test Loss: 0.0005, Accuracy: 9902/10000
LeNet-5 - Epoch 8: Test Loss: 0.0008, Accuracy: 9857/10000
LeNet-5 - Epoch 9: Test Loss: 0.0006, Accuracy: 9885/10000
LeNet-5 - Epoch 10: Test Loss: 0.0009, Accuracy: 9836/10000


#AlexNet

In [ ]:
# MNIST dataset (images and labels)
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize to fit AlexNet input dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader (input pipeline)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


In [ ]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct


In [ ]:
# Example for AlexNet
model = AlexNet().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, correct = test(model, device, test_loader, criterion)
    print(f"AlexNet - Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


AlexNet - Epoch 1: Test Loss: 0.0010, Accuracy: 9811/10000
AlexNet - Epoch 2: Test Loss: 0.0011, Accuracy: 9810/10000
AlexNet - Epoch 3: Test Loss: 0.0008, Accuracy: 9862/10000
AlexNet - Epoch 4: Test Loss: 0.0012, Accuracy: 9855/10000
AlexNet - Epoch 5: Test Loss: 0.0005, Accuracy: 9902/10000
AlexNet - Epoch 6: Test Loss: 0.0006, Accuracy: 9890/10000
AlexNet - Epoch 7: Test Loss: 0.0006, Accuracy: 9895/10000
AlexNet - Epoch 8: Test Loss: 0.0004, Accuracy: 9921/10000
AlexNet - Epoch 9: Test Loss: 0.0006, Accuracy: 9905/10000
AlexNet - Epoch 10: Test Loss: 0.0004, Accuracy: 9916/10000


#VGGNet

In [ ]:
# MNIST dataset (images and labels)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit VGGNet input dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader (input pipeline)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Additional convolutional layers can be added here
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(128 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


In [ ]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct


In [ ]:
model = VGGNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 11):  # 10 epochs
    train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, correct = test(model, device, test_loader, criterion)
    print(f"Epoch {epoch}: Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}")


Epoch 1: Test Loss: 0.0014, Accuracy: 9719/10000
Epoch 2: Test Loss: 0.0008, Accuracy: 9825/10000
Epoch 3: Test Loss: 0.0007, Accuracy: 9853/10000
Epoch 4: Test Loss: 0.0008, Accuracy: 9845/10000
Epoch 5: Test Loss: 0.0009, Accuracy: 9817/10000
Epoch 6: Test Loss: 0.0007, Accuracy: 9866/10000
Epoch 7: Test Loss: 0.0006, Accuracy: 9893/10000
Epoch 8: Test Loss: 0.0007, Accuracy: 9868/10000
Epoch 9: Test Loss: 0.0005, Accuracy: 9904/10000
